In [2]:
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [42]:
filterquery = {"$and":
    [
     {"$or":
        [
            {"category_code": "software"},
            {"category_code": "web"},
            {"category_code": "mobile"},
            {"category_code": "games_video"}
        ]},
      {"number_of_employees": {"$gt":10}},
      {"$and":
         [
             {"founded_year": {"$gt":2010}},
             {"deadpooled_year":{"$ne": None}},
             {"deadpooled_year":{"$exists":True}}
         ]},
      {"$and":
         [
             {"$or":
                 [
                     {"relationships.title":{'$regex':'[Dd]esigner'}},
                     {"relationships.title":{'$regex':'[Cc]reative'}},
                     {"relationships.title":{'$regex':'U[XI]'}},
                     {"relationships.title":{'$regex':'[Cc]reative'}},
                 ]},
             {"relationships.title":{'$regex':'[Dd]evelopers'}}
        ]
    }
    ]
}
cursor = db.companies.find(filterquery)
# print(list(map(lambda r: r["name"],cursor)))
db.companies.count(filterquery)


# {"relationships.title":/Executive/}

/home/luis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


0

### I suposse that startup is a company that have less than 10

In [ ]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

def convertRestaurant(restaurant):
    return {
        "name": restaurant["name"],
        "cuisine":restaurant["cuisine"],
        "position": toGeoJSON(restaurant["address"]["coord"])
    }

cursor = db.restaurants.find({"address.coord":{"$size":2}},{"_id":0})
geoRestaurants = list(map(lambda r: convertRestaurant(r),cursor))
